In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test
from Models.basic_models import average_model
from Models.two_layer_concat import two_layer_concat

set_figsize(20, 15)

Training directory found, 36 series
Validation directory found, 6 series
Testing directory found, 10 series


In [2]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)


validation = TAVR_3_Frame("__valid")
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train")
train_loader = tavr_dataloader(training, batch_size=8, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_concat(standardize_slice=True)
loss_fn = batch_l2_loss()

using device: cpu


In [ ]:
learning_rate = 1e-5
momentum = 0.90
reg = 1e-7

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=reg, nesterov=True)

In [ ]:
train(model, optimizer, train_loader, val_loader, loss_fn, device, 
         epochs=3, print_every=30, print_level=3, lr_decay=0.8)


****Epoch 0 Iteration 0, loss = 1332.8939
Validation loss 1164.0092 over 81 frames
conv_a1.weight,   	norm: 4.0379e+00, 	update norm: 2.3402e-02 	Update/norm: 5.7955e-03
conv_a1.bias,   	norm: 3.1126e-01, 	update norm: 1.4387e-04 	Update/norm: 4.6222e-04
conv_b1.weight,   	norm: 3.5187e+00, 	update norm: 2.3325e-02 	Update/norm: 6.6288e-03
conv_b1.bias,   	norm: 2.6548e-01, 	update norm: 1.4539e-04 	Update/norm: 5.4765e-04
final.weight,   	norm: 1.3835e+00, 	update norm: 3.9827e-02 	Update/norm: 2.8788e-02
final.bias,   	norm: 1.3720e-01, 	update norm: 9.8109e-05 	Update/norm: 7.1508e-04

Iter 0... ... ... 
Iter 10... ... ... 
Iter 20... ... ... 
Iteration 30, loss = 98.1833


In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, val_loader, loss_fn, device)
test(ave_model, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)